In [1]:
import pandas as pd
import json
from os import path
from collections import defaultdict
import numpy as np
import geopy.distance

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from score import*
import time

In [2]:
route_data_df = pd.read_json("./model_build_inputs/route_data.json", orient='index') # shape: (6, 6112)
actual_sequences_df = pd.read_json("./model_build_inputs/actual_sequences.json", orient='index') # shape: (6, 6112)

# merge actual sequences (solutions) to df

route_actual_sequence_data_df = (pd.merge(route_data_df, actual_sequences_df, left_index=True, right_index=True, 
                                          how = "inner"))
route_actual_sequence_data_df["numeric_route_score"] = route_actual_sequence_data_df.route_score.map({"High":1,
                                         "Medium":1,"Low":1})

Complete route data with actual solutions

In [3]:
route_actual_sequence_data_df.head()

,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops,actual,numeric_route_score
RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,"{'AD': {'lat': 34.099611, 'lng': -118.283062, ...","{'AD': 105, 'AF': 47, 'AG': 4, 'BA': 33, 'BE':...",1
RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,DSE4,2018-07-28,15:44:41,4247527.0,High,"{'AC': {'lat': 47.689446, 'lng': -122.296071, ...","{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'...",1
RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,DSE5,2018-08-18,15:32:04,4247527.0,High,"{'AA': {'lat': 47.268001, 'lng': -122.5079, 't...","{'AA': 58, 'AD': 94, 'AJ': 41, 'AP': 124, 'AR'...",1
RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,DLA9,2018-08-15,15:09:38,3313071.0,High,"{'AB': {'lat': 33.823076, 'lng': -118.058727, ...","{'AB': 122, 'AC': 39, 'AG': 27, 'AI': 16, 'AU'...",1
RouteID_0021a2aa-780f-460d-b09a-f301709e2523,DLA7,2018-08-05,14:23:26,3313071.0,High,"{'AA': {'lat': 33.84364, 'lng': -117.773651, '...","{'AA': 43, 'AB': 54, 'AF': 72, 'AG': 31, 'AI':...",1


Apply transformations to append a new column with zone sequences

In [4]:
def sequence_of_zones(row):
    sequence = []
    actual = sorted(list(row["actual"]), key=lambda l: row["actual"][l])
    actual.append(actual[0])
    
    # finding the sequence of zones
    for i, stop in enumerate(actual):
        if len(sequence) == 0 or i == len(actual)-1:
            sequence.append("0")
        elif row["stops"][stop]["zone_id"] is None:
            continue
        elif row["stops"][stop]["zone_id"] != sequence[-1]:
            sequence.append(row["stops"][stop]["zone_id"])
        # drop the stop if the stop has no zone id
   
    return sequence

def num_stops_in_zone(row):
    num_stops = {}
    all_stops = row["stops"]
    for stop in all_stops:
        # getting the zone
        zone_id = all_stops[stop]["zone_id"]
        if zone_id not in num_stops:
            num_stops[zone_id] = 1
        else:
            num_stops[zone_id] += 1
    
    return num_stops

route_actual_sequence_data_df["sequences_of_zones"] = route_actual_sequence_data_df.apply(lambda row: sequence_of_zones(row), 
                                                        axis = 1)

route_actual_sequence_data_df.head()

,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops,actual,numeric_route_score,sequences_of_zones
RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,"{'AD': {'lat': 34.099611, 'lng': -118.283062, ...","{'AD': 105, 'AF': 47, 'AG': 4, 'BA': 33, 'BE':...",1,"[0, A-2.2A, A-2.1A, A-1.1A, A-1.2A, A-1.3A, A-..."
RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,DSE4,2018-07-28,15:44:41,4247527.0,High,"{'AC': {'lat': 47.689446, 'lng': -122.296071, ...","{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'...",1,"[0, D-3.2A, D-3.1A, D-2.1A, D-2.2A, D-2.3A, D-..."
RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,DSE5,2018-08-18,15:32:04,4247527.0,High,"{'AA': {'lat': 47.268001, 'lng': -122.5079, 't...","{'AA': 58, 'AD': 94, 'AJ': 41, 'AP': 124, 'AR'...",1,"[0, A-2.2E, A-2.1E, A-2.1D, A-2.2D, A-2.3D, A-..."
RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,DLA9,2018-08-15,15:09:38,3313071.0,High,"{'AB': {'lat': 33.823076, 'lng': -118.058727, ...","{'AB': 122, 'AC': 39, 'AG': 27, 'AI': 16, 'AU'...",1,"[0, B-11.1G, B-11.1H, B-11.2H, B-12.1G, B-12.3..."
RouteID_0021a2aa-780f-460d-b09a-f301709e2523,DLA7,2018-08-05,14:23:26,3313071.0,High,"{'AA': {'lat': 33.84364, 'lng': -117.773651, '...","{'AA': 43, 'AB': 54, 'AF': 72, 'AG': 31, 'AI':...",1,"[0, C-17.3D, C-17.2D, C-17.1D, C-17.1E, C-17.2..."


In [5]:
route_actual_sequence_data_df["route_score"].value_counts()

Medium    3292
High      2718
Low        102
Name: route_score, dtype: int64

## Training

```
High   - 2174 - 544  
Medium - 2634 - 658  
Low    -   82 -  20
```

## Create Training Set

In [6]:
train_high = route_actual_sequence_data_df.loc[route_actual_sequence_data_df["route_score"] == "High"][:2174]
train_medium = route_actual_sequence_data_df.loc[route_actual_sequence_data_df["route_score"] == "Medium"][:2634]
train_low = route_actual_sequence_data_df.loc[route_actual_sequence_data_df["route_score"] == "Low"][:82]
train_df = pd.concat([train_high, train_medium, train_low])
train_df.head()

,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops,actual,numeric_route_score,sequences_of_zones
RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,"{'AD': {'lat': 34.099611, 'lng': -118.283062, ...","{'AD': 105, 'AF': 47, 'AG': 4, 'BA': 33, 'BE':...",1,"[0, A-2.2A, A-2.1A, A-1.1A, A-1.2A, A-1.3A, A-..."
RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,DSE4,2018-07-28,15:44:41,4247527.0,High,"{'AC': {'lat': 47.689446, 'lng': -122.296071, ...","{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'...",1,"[0, D-3.2A, D-3.1A, D-2.1A, D-2.2A, D-2.3A, D-..."
RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,DSE5,2018-08-18,15:32:04,4247527.0,High,"{'AA': {'lat': 47.268001, 'lng': -122.5079, 't...","{'AA': 58, 'AD': 94, 'AJ': 41, 'AP': 124, 'AR'...",1,"[0, A-2.2E, A-2.1E, A-2.1D, A-2.2D, A-2.3D, A-..."
RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,DLA9,2018-08-15,15:09:38,3313071.0,High,"{'AB': {'lat': 33.823076, 'lng': -118.058727, ...","{'AB': 122, 'AC': 39, 'AG': 27, 'AI': 16, 'AU'...",1,"[0, B-11.1G, B-11.1H, B-11.2H, B-12.1G, B-12.3..."
RouteID_0021a2aa-780f-460d-b09a-f301709e2523,DLA7,2018-08-05,14:23:26,3313071.0,High,"{'AA': {'lat': 33.84364, 'lng': -117.773651, '...","{'AA': 43, 'AB': 54, 'AF': 72, 'AG': 31, 'AI':...",1,"[0, C-17.3D, C-17.2D, C-17.1D, C-17.1E, C-17.2..."


## Create Test Set

In [7]:
test_df = route_actual_sequence_data_df.loc[route_actual_sequence_data_df["route_score"] == "High"][2174:]

# small test
test_df = test_df[:20]

test_df.head()

,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops,actual,numeric_route_score,sequences_of_zones
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,2018-08-14,16:39:35,3313071.0,High,"{'AF': {'lat': 33.892069, 'lng': -118.106502, ...","{'AF': 118, 'AL': 165, 'AU': 3, 'AY': 104, 'BC...",1,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-..."
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,2018-07-20,15:59:32,4247527.0,High,"{'AF': {'lat': 47.680001, 'lng': -122.27859, '...","{'AF': 91, 'AL': 22, 'AM': 25, 'AO': 100, 'AQ'...",1,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-..."
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,2018-08-09,16:08:12,3313071.0,High,"{'AC': {'lat': 34.084802, 'lng': -118.250983, ...","{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'...",1,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-..."
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,2018-08-05,14:09:23,4247527.0,High,"{'AA': {'lat': 30.191954, 'lng': -97.901269, '...","{'AA': 169, 'AC': 206, 'AH': 43, 'AJ': 126, 'A...",1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2..."
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,2018-08-10,15:32:56,4247527.0,High,"{'AB': {'lat': 47.685754, 'lng': -122.306757, ...","{'AB': 37, 'AE': 59, 'AN': 66, 'AR': 68, 'AT':...",1,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-..."


From train_df, create dictionaries with key = station_code, value = all_zones_ordered  
(Ordered zone is necessary when indexing, during construction of transition matrix)

In [8]:
station_sequences = defaultdict(list)
all_zones = defaultdict(set)

for id, row in train_df.iterrows():
    station_sequences[row["station_code"]].append((row["sequences_of_zones"],row['numeric_route_score']))
    all_zones[row["station_code"]] |= set(row["sequences_of_zones"])
all_zones_ordered = {k: {zone:id for id, zone in enumerate(sorted(list(v)))} for k, v in all_zones.items()}

Build frequency matrix for each station_code

In [9]:
zone_frequency_matrices = {}
for k, v in all_zones_ordered.items():
    zone_frequency_matrices[k] = np.zeros((len(v), len(v)))

for station_code, list_of_sequences in station_sequences.items():
    for tuples in list_of_sequences:
        sequence = tuples[0]
        for i in range(len(sequence)-1):
            zone_pos_cur = all_zones_ordered[station_code][sequence[i]]
            zone_pos_next = all_zones_ordered[station_code][sequence[i+1]]
            if zone_pos_cur == zone_pos_next:
                print(sequence)
                print("error!",i, zone_pos_cur, zone_pos_next)
            zone_frequency_matrices[station_code][zone_pos_cur][zone_pos_next] += tuples[1]

In [10]:
for station_code, freq_mat in zone_frequency_matrices.items():
    print(station_code, freq_mat.shape)

DLA3 (1346, 1346)
DSE4 (1697, 1697)
DSE5 (1216, 1216)
DLA9 (4559, 4559)
DLA7 (4148, 4148)
DCH4 (2209, 2209)
DCH3 (1754, 1754)
DCH2 (1165, 1165)
DBO2 (2140, 2140)
DLA4 (1015, 1015)
DLA8 (3453, 3453)
DBO3 (3219, 3219)
DAU1 (2361, 2361)
DSE2 (529, 529)
DCH1 (1718, 1718)
DBO1 (529, 529)
DLA5 (1271, 1271)


In [11]:
all_zones = set()

for station_code, list_of_sequences in station_sequences.items():
    for tuples in list_of_sequences:
        sequence = tuples[0]
        all_zones.update(sequence)

In [12]:
len(all_zones)

8688

## Functions

In [13]:
def arc_diff(a, b):
    stop_set_a = set()
    for i in range(len(a)-1):
        stop_set_a.add((a[i], a[i+1]))

    stop_set_b = set()
    for i in range(len(b)-1):
        stop_set_b.add((b[i], b[i+1]))

    return 100*(len(stop_set_b.difference(stop_set_a))/len(stop_set_a))

In [14]:
def uniq_seq_transform(seq): # order preserving
    seen = set()
    return [x for x in seq if x not in seen and not seen.add(x)]

In [15]:
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = np.zeros((len(locations),len(locations)  ))
    for from_counter, from_node in enumerate(locations):
        # distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                distances[from_counter][to_counter] = 100*geopy.distance.distance(from_node,to_node).km
    return distances

In [16]:
def solve_TSP(instance, probability_mat):
    
    def create_data_model(prob_mat):
        """Stores the data for the problem."""
        # Note that distances SHOULD BE integers; multiply by 100 for probabilities
        data = {}
        data['distance_matrix'] = prob_mat
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def get_routes(solution, routing, manager):
          """Get vehicle routes from a solution and store them in an array."""
          # Get vehicle routes and store them in a two dimensional array whose
          # i,j entry is the jth location visited by vehicle i along its route.
          routes = []
          for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
              index = solution.Value(routing.NextVar(index))
              route.append(manager.IndexToNode(index))
            routes.append(route)
            return routes
    
    data = create_data_model(probability_mat)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Register callback with the solver.
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
#     # Setting search strategy
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.local_search_metaheuristic = (
#         routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 10
#     search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    temp_sol = get_routes(solution, routing, manager)
           
    return [instance[i] for i in temp_sol[0][:-1]] + [instance[0]]

In [17]:
def create_zone_dist_mat(row, uniq_seq):
    
    # get station location
    station_stop = [k for k, v in row['actual'].items() if v == 0]
    station_loc = (row['stops'][station_stop[0]]['lat'], row['stops'][station_stop[0]]['lng'])
    
    # get zone locations   
    zone_mapping = []
    for k, stop in row['stops'].items():
        zone_mapping.append({'zone_id':stop['zone_id'], 'coordinate':(stop['lat'],stop['lng'])})
    zone_mapping = pd.DataFrame(zone_mapping)
    zone_mapping = zone_mapping.groupby('zone_id').agg({'coordinate':lambda lst: list(set(lst.tolist()))})
    zone_mapping = zone_mapping.explode('coordinate').reset_index()
    zone_mapping.loc[:,'lat']= zone_mapping.coordinate.map(lambda x:x[0])
    zone_mapping.loc[:,'lng']= zone_mapping.coordinate.map(lambda x:x[1])
    df = zone_mapping.groupby('zone_id').agg({'lat':'mean','lng':'mean'}).reset_index()
    df['coordinate'] = list(zip(df.lat, df.lng))
    
    zone_locations = [df.loc[df['zone_id'] == zone, 'coordinate'].iloc[0] for zone in uniq_seq[1:-1]]
    zone_locations = [station_loc] + zone_locations
            
    # build zone_dist_mat
    return compute_euclidean_distance_matrix(zone_locations)

In [18]:
def build_zone_fm(uniq_seq, all_zones_ordered, zone_frequency_matrices):
    
    # build transition matrix (sum of submatrices from all stations)
    zone_fm_sum = np.array([[0 for i in uniq_seq[:-1]] for i in uniq_seq[:-1]])

    for station, freq_mat in zone_frequency_matrices.items():
        idxs = [all_zones_ordered[station][s] if s in all_zones_ordered[station] else -1 for s in uniq_seq[:-1]]

        zone_freq_sub_matrix = np.array([[0 for i in idxs] for i in idxs])
        for j_id, j in enumerate(idxs):
            for k_id, k in enumerate(idxs):
                if j == -1 or k == -1:
                    zone_freq_sub_matrix[j_id][k_id] = 0
                else:
                    zone_freq_sub_matrix[j_id][k_id] = zone_frequency_matrices[station][j][k]

        zone_fm_sum += zone_freq_sub_matrix

    if zone_fm_sum.sum() == 0:
        print('No hist info for instance', i)
        
    return zone_fm_sum

## Distance-based (zone sequence)

Predict zone sequences by solving TSP using zone (absolute) distance matrix

In [19]:
start = time.time()

pred_sequences = []
arc_diffs = []
zone_score = []
trans_count = 0

for index, row in test_df.iterrows():
    seq = row['sequences_of_zones']
    
    # get unique zone sequence
    uniq_seq = uniq_seq_transform(seq)
    sorted_zones = sorted(uniq_seq)
    sorted_zones.append('0')
    uniq_seq.append('0')

    zone_dist_mat = create_zone_dist_mat(row, sorted_zones)
    
    predicted = solve_TSP(sorted_zones, zone_dist_mat)
    
    pred_sequences.append(predicted)
    arc_diffs.append(arc_diff(uniq_seq, predicted))
    
    # compute zone score
    mat_dict = dict.fromkeys(sorted_zones[:-1])
    for k in range(len(sorted_zones[:-1])):
        mat_dict[sorted_zones[k]] = dict(zip(sorted_zones[:-1], zone_dist_mat[k]))
    zone_score.append(score(uniq_seq, predicted, mat_dict))
    
    # transform counter
    trans_count += not len(seq) == len(uniq_seq)
    
end = time.time()

In [20]:
dist_df = test_df.assign(pred_sequences=pred_sequences, AD=arc_diffs,
                              zone_score=zone_score, time=end-start)[['station_code', 'sequences_of_zones', 
                                                                      'pred_sequences', 'AD', 'zone_score', 'time']]
dist_df

,station_code,sequences_of_zones,pred_sequences,AD,zone_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...","[0, F-4.3B, F-4.2B, F-4.1B, F-4.2C, F-4.3C, F-...",93.333333,0.041023,2.405951
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-...","[0, D-5.1B, D-5.1C, D-5.2C, D-5.3C, D-5.3D, D-...",72.727273,0.008896,2.405951
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...","[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-7.2A, A-...",78.260870,0.025279,2.405951
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2...","[0, B-12.1H, B-12.1G, B-12.3E, B-12.3G, B-12.2...",75.000000,0.055791,2.405951
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...","[0, D-1.2H, D-1.3J, D-1.2J, D-2.1J, D-1.1J, D-...",75.000000,0.019273,2.405951
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-...","[0, H-1.3B, G-1.3A, G-1.3B, G-1.1B, G-1.2B, G-...",76.923077,0.033160,2.405951
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1...","[0, C-12.2H, C-12.1H, C-12.3G, C-12.2G, C-12.1...",45.000000,0.090060,2.405951
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...","[0, J-20.1C, J-20.1B, J-20.3B, J-20.2B, J-20.2...",60.000000,0.112222,2.405951
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...","[0, A-7.1B, A-7.3C, A-7.3D, A-7.2D, A-7.2C, A-...",72.727273,0.066293,2.405951
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3...","[0, E-16.1A, G-16.1A, G-16.2A, G-16.3A, G-16.3...",87.500000,0.083720,2.405951


In [21]:
dist_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\2328375253.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dist_df.mean()


AD            71.987573
zone_score     0.058118
time           2.405951
dtype: float64

## Markov Counting - for comparison

Training with all stations  
Constuction of transition matrix is not station-specific

In [22]:
start = time.time()

pred_sequences = []
arc_diffs = []
zone_score = []
solve_counter = 0

for index, row in test_df.iterrows():

    seq = row['sequences_of_zones']
    
    # get unique zone sequence
    uniq_seq = uniq_seq_transform(seq)
    sorted_zones = sorted(uniq_seq)
    sorted_zones.append('0')
    uniq_seq.append('0')
    
    # build transition matrix (sum of submatrices from all stations)
    zone_freq_mat = build_zone_fm(sorted_zones, all_zones_ordered, zone_frequency_matrices)
    
    # normalize    
    zone_freq_mat = zone_freq_mat + 1 # smoothing
    trans_prob_mat = zone_freq_mat/zone_freq_mat.sum(axis=1, keepdims=True)

    # --- for scoring ---
    
    zone_dist_mat = create_zone_dist_mat(row, sorted_zones)
    
    # --- --- --- --- --- 
    
    predicted = solve_TSP(sorted_zones, 1000*(-np.log(trans_prob_mat))) # multiply 
    
    pred_sequences.append(predicted)
    arc_diffs.append(arc_diff(uniq_seq, predicted))
    
    # compute zone score
    mat_dict = dict.fromkeys(sorted_zones[:-1])
    for k in range(len(sorted_zones[:-1])):
        mat_dict[sorted_zones[k]] = dict(zip(sorted_zones[:-1], zone_dist_mat[k]))
    zone_score.append(score(uniq_seq, predicted, mat_dict))
    
    print("Solving", solve_counter, end='\r')
    solve_counter += 1
    
end = time.time()

In [23]:
markov_all_df = test_df.assign(pred_sequences=pred_sequences, AD=arc_diffs,
                                    zone_score=zone_score, time=end-start)[['station_code', 'sequences_of_zones', 'pred_sequences', 'AD', 'zone_score', 'time']]
markov_all_df

,station_code,sequences_of_zones,pred_sequences,AD,zone_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...","[0, F-4.2B, F-4.1D, F-4.2D, F-4.3D, F-4.3C, F-...",33.333333,0.064731,2.550212
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-...","[0, D-5.3B, D-5.2B, D-5.1B, D-5.1C, D-5.2C, D-...",100.000000,0.000000,2.550212
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...","[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...",0.000000,0.000000,2.550212
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2...","[0, B-12.1H, B-12.1G, B-12.2G, B-12.3G, B-12.3...",100.000000,0.017316,2.550212
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...","[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...",0.000000,0.000000,2.550212
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-...","[0, H-1.1A, H-1.2A, H-1.3A, H-1.3B, G-1.2A, G-...",53.846154,0.014304,2.550212
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1...","[0, C-11.3E, C-11.3G, C-11.2G, C-11.1G, C-11.1...",45.000000,0.037230,2.550212
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...","[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...",0.000000,0.000000,2.550212
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...","[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...",0.000000,0.000000,2.550212
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3...","[0, E-16.1B, E-16.2B, E-16.3B, E-16.3A, E-16.2...",62.500000,0.095985,2.550212


In [24]:
markov_all_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\1497858028.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  markov_all_df.mean()


AD            38.651195
zone_score     0.025556
time           2.550212
dtype: float64

## SOP - Stage 1

```
Needed:

1. Zone distance matrix of training instance
2. Freq matrix for all stations (again, using only training instances)
3. Assign weights
```

In [25]:
def create_adj_mat(mat_dim, route_sol):
    adj_mat = np.zeros((mat_dim, mat_dim))
    
    for i in range(len(route_sol)-1):
        adj_mat[route_sol[i]][route_sol[i+1]] += 1
            
    return adj_mat

Train

In [26]:
# initialization

w1 = 1
w2 = 1

In [27]:
i = 0
for j in range(1): # num of iterations
    
    for index, row in train_df[:100].iterrows():
        seq = row['sequences_of_zones']

        # get unique zone sequence
        uniq_seq = uniq_seq_transform(seq)
        randomized_zones = sorted(uniq_seq)

        uniq_seq.append('0')
        randomized_zones.append('0')

        # TRANSITION MATRIX

        # build transition matrix (sum of submatrices from all stations)
        zone_freq_mat = build_zone_fm(randomized_zones, all_zones_ordered, zone_frequency_matrices)
        zone_freq_mat = zone_freq_mat + 1 # smoothing

        # normalize    
        trans_prob_mat = zone_freq_mat/zone_freq_mat.sum(axis=1, keepdims=True)
        trans_prob_mat = -np.log(trans_prob_mat)
#         trans_prob_mat = trans_prob_mat/trans_prob_mat.sum(axis=1, keepdims=True)

        # DISTANCE MATRIX

        zone_dist_mat = create_zone_dist_mat(row, randomized_zones)
        zone_dist_mat = zone_dist_mat + 1 # smoothing

        # normalize
#         dist_prob_mat = zone_dist_mat/zone_dist_mat.sum(axis=1, keepdims=True)
#         dist_prob_mat = softmax(-zone_dist_mat, axis=1)
        dist_prob_mat = 1/zone_dist_mat
        dist_prob_mat = dist_prob_mat/dist_prob_mat.sum(axis=1, keepdims=True)
        dist_prob_mat = -np.log(dist_prob_mat)
#         dist_prob_mat = dist_prob_mat/dist_prob_mat.sum(axis=1, keepdims=True)
        
        
        # COMBINATION MATRIX
        
        combi_mat = dist_prob_mat*w1 + trans_prob_mat*w2

        # solve
        predicted = solve_TSP(randomized_zones, 10000*combi_mat)
        
        act_adj = create_adj_mat(len(combi_mat), [i for i in range(len(uniq_seq[:-1]))])
        
        # pred_zone_order to rearrange order of prob and dist matrices to correspond with uniq_seq
        pred_zone_order = [randomized_zones.index(zone) for zone in predicted[:-1]]
        act_zone_order = [randomized_zones.index(zone) for zone in uniq_seq[:-1]]
        
        pred_adj = create_adj_mat(len(combi_mat), pred_zone_order)

#         # update weights
#         w1 = w1 + 1*(np.multiply(act_adj, dist_prob_mat).sum() 
#                      - np.multiply(pred_adj, dist_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum())
#         w2 = w2 + 1*(np.multiply(act_adj, trans_prob_mat).sum() 
#                      - np.multiply(pred_adj, trans_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum())
        
#         # update weights
#         w1 = w1 + 0.000001*(np.multiply(act_adj, dist_prob_mat[np.ix_(act_zone_order, act_zone_order)]).sum() 
#                      - np.multiply(pred_adj, dist_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum())
#         w2 = w2 + 0.000001*(np.multiply(act_adj, trans_prob_mat[np.ix_(act_zone_order, act_zone_order)]).sum() 
#                      - np.multiply(pred_adj, trans_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum())
        
        # update weights (reverse)
        w1 = w1 + 0.00001*(np.multiply(pred_adj, dist_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum()
                            - np.multiply(act_adj, dist_prob_mat[np.ix_(act_zone_order, act_zone_order)]).sum()) 
        w2 = w2 + 0.00001*(np.multiply(pred_adj, trans_prob_mat[np.ix_(pred_zone_order, pred_zone_order)]).sum()
                            - np.multiply(act_adj, trans_prob_mat[np.ix_(act_zone_order, act_zone_order)]).sum()) 
        
        i += 1
        
        print(i, w1, w2, end='\r')

In [28]:
w1

1.0116787638340274

In [29]:
w2

1.0275912318753342

In [30]:
# set beta (w1) value

beta = [w1/(w1+w2)]
beta

[0.49609848914690385]

In [31]:
trans_prob_mat[0]

array([3.49650756, 3.49650756, 2.80336038, 2.39789527, 2.39789527,
       2.80336038, 2.39789527, 3.49650756, 2.39789527, 2.80336038,
       3.49650756, 2.39789527, 2.80336038, 2.39789527, 3.49650756,
       2.80336038])

In [32]:
dist_prob_mat[0]

array([5.18468041e-03, 7.97029718e+00, 7.95848697e+00, 7.99305064e+00,
       7.98917186e+00, 7.95566871e+00, 7.96768636e+00, 7.95494567e+00,
       7.97676230e+00, 7.99110226e+00, 7.96653827e+00, 7.96974286e+00,
       7.96266846e+00, 7.97315110e+00, 7.98475377e+00, 7.97739693e+00])

Test

In [33]:
for beta_param in [float(i) for i in beta]:

    pred_sequences = []
    arc_diffs = []
    zone_score = []

    for index, row in test_df.iterrows():
        seq = row['sequences_of_zones']

        # get unique zone sequence
        uniq_seq = uniq_seq_transform(seq)
        randomized_zones = sorted(uniq_seq)

        uniq_seq.append('0')
        randomized_zones.append('0')

        # TRANSITION MATRIX

        # build transition matrix (sum of submatrices from all stations)
        zone_freq_mat = build_zone_fm(randomized_zones, all_zones_ordered, zone_frequency_matrices)
        zone_freq_mat = zone_freq_mat + 1 # smoothing

        # normalize    
        trans_prob_mat = zone_freq_mat/zone_freq_mat.sum(axis=1, keepdims=True)
        trans_prob_mat = -np.log(trans_prob_mat)
#         trans_prob_mat = trans_prob_mat/trans_prob_mat.sum(axis=1, keepdims=True)

        # DISTANCE MATRIX

        zone_dist_mat = create_zone_dist_mat(row, randomized_zones)
        zone_dist_mat = zone_dist_mat + 1 # smoothing

        # normalize
#         dist_prob_mat = zone_dist_mat/zone_dist_mat.sum(axis=1, keepdims=True)
#         dist_prob_mat = softmax(-zone_dist_mat, axis=1)
        dist_prob_mat = 1/zone_dist_mat
#         dist_prob_mat = dist_prob_mat/dist_prob_mat.sum(axis=1, keepdims=True)
        dist_prob_mat = -np.log(dist_prob_mat)
#         dist_prob_mat = dist_prob_mat/dist_prob_mat.sum(axis=1, keepdims=True)
        
        # COMBINATION MATRIX
        
        combi_mat = dist_prob_mat*(1-beta_param) + trans_prob_mat*(beta_param)

        # solve
        predicted = solve_TSP(randomized_zones, 10000*combi_mat)

        pred_sequences.append(predicted)
        arc_diffs.append(arc_diff(uniq_seq, predicted))

        # compute zone score
        mat_dict = dict.fromkeys(randomized_zones[:-1])
        for k in range(len(randomized_zones[:-1])):
            mat_dict[randomized_zones[k]] = dict(zip(randomized_zones[:-1], zone_dist_mat[k]))
        zone_score.append(score(uniq_seq, predicted, mat_dict))

In [34]:
SOP_stage1_df = test_df.assign(pred_sequences=pred_sequences, AD=arc_diffs,
                             score=zone_score)[['station_code', 'sequences_of_zones', 'pred_sequences', 'AD', 'score']]
SOP_stage1_df

,station_code,sequences_of_zones,pred_sequences,AD,score
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...","[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...",0.000000,0.000000
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-...","[0, D-5.3B, D-5.2B, D-5.1B, D-5.1C, D-5.2C, D-...",100.000000,0.000000
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...","[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...",0.000000,0.000000
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2...","[0, B-12.1H, B-12.1G, B-12.2G, B-12.3G, B-12.3...",100.000000,0.017316
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...","[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...",0.000000,0.000000
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-...","[0, H-1.1A, H-1.2A, H-1.3A, H-1.3B, G-1.1A, G-...",38.461538,0.006269
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1...","[0, C-12.3G, C-12.2G, C-12.1G, C-12.1H, C-12.2...",35.000000,0.060835
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...","[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...",0.000000,0.000000
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...","[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...",0.000000,0.000000
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3...","[0, E-16.1B, E-16.2B, E-16.3B, E-16.3A, E-16.2...",62.500000,0.095985


In [35]:
SOP_stage1_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\3767397974.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  SOP_stage1_df.mean()


AD       35.256965
score     0.021107
dtype: float64

## STAGE TWO: Routing of stops

In [36]:
test_df

,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops,actual,numeric_route_score,sequences_of_zones
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,2018-08-14,16:39:35,3313071.0,High,"{'AF': {'lat': 33.892069, 'lng': -118.106502, ...","{'AF': 118, 'AL': 165, 'AU': 3, 'AY': 104, 'BC...",1,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-..."
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,2018-07-20,15:59:32,4247527.0,High,"{'AF': {'lat': 47.680001, 'lng': -122.27859, '...","{'AF': 91, 'AL': 22, 'AM': 25, 'AO': 100, 'AQ'...",1,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-..."
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,2018-08-09,16:08:12,3313071.0,High,"{'AC': {'lat': 34.084802, 'lng': -118.250983, ...","{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'...",1,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-..."
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,2018-08-05,14:09:23,4247527.0,High,"{'AA': {'lat': 30.191954, 'lng': -97.901269, '...","{'AA': 169, 'AC': 206, 'AH': 43, 'AJ': 126, 'A...",1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2..."
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,2018-08-10,15:32:56,4247527.0,High,"{'AB': {'lat': 47.685754, 'lng': -122.306757, ...","{'AB': 37, 'AE': 59, 'AN': 66, 'AR': 68, 'AT':...",1,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-..."
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,2018-08-12,13:45:00,3313071.0,High,"{'AE': {'lat': 34.188731, 'lng': -118.411372, ...","{'AE': 5, 'AP': 70, 'AS': 27, 'AV': 74, 'AX': ...",1,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-..."
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,2018-07-29,12:15:00,4247527.0,High,"{'AA': {'lat': 41.75077, 'lng': -71.291834, 't...","{'AA': 30, 'AH': 29, 'AJ': 44, 'AL': 129, 'AQ'...",1,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1..."
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,2018-08-08,15:43:59,3313071.0,High,"{'AD': {'lat': 33.73724, 'lng': -117.882531, '...","{'AD': 152, 'AF': 147, 'AG': 94, 'AM': 189, 'A...",1,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2..."
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,2018-08-12,15:53:04,3313071.0,High,"{'AB': {'lat': 34.084199, 'lng': -118.253285, ...","{'AB': 35, 'AD': 45, 'AG': 106, 'AI': 96, 'AN'...",1,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-..."
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,2018-08-02,14:32:42,3313071.0,High,"{'AG': {'lat': 34.201195, 'lng': -118.381722, ...","{'AG': 45, 'AL': 114, 'AO': 68, 'AQ': 119, 'AX...",1,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3..."


### Load travel times

In [37]:
def create_time_matrix(stopslist, jsondata):
    times = np.zeros((len(stopslist), len(stopslist)))
    for i, source in enumerate(stopslist):
        for j, dest in enumerate(stopslist):
            times[i][j] = jsondata[source][dest]
    return times

In [38]:
with open("./model_build_inputs/travel_times.json") as f:
    travel_times_dict = json.load(f)

In [39]:
def solve_TSP_LS(instance, probability_mat):
    
    def create_data_model(prob_mat):
        """Stores the data for the problem."""
        # Note that distances SHOULD BE integers; multiply by 100 for probabilities
        data = {}
        data['distance_matrix'] = prob_mat
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def get_routes(solution, routing, manager):
          """Get vehicle routes from a solution and store them in an array."""
          # Get vehicle routes and store them in a two dimensional array whose
          # i,j entry is the jth location visited by vehicle i along its route.
          routes = []
          for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
              index = solution.Value(routing.NextVar(index))
              route.append(manager.IndexToNode(index))
            routes.append(route)
            return routes
    
    data = create_data_model(probability_mat)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Register callback with the solver.
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

#     # Setting first solution heuristic.
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Setting search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    temp_sol = get_routes(solution, routing, manager)
           
    return [instance[i] for i in temp_sol[0][:-1]] + [instance[0]]

### Traditional TSP solution

In [40]:
start = time.time()

stops_score = []
actual_routes = []
pred_routes = []
solve_counter = 0

for index, row in test_df.iterrows():
    stops_list = [*{k: v for k, v in sorted(row["actual"].items(), key=lambda item: item[1])}]
    actual_route = stops_list + [stops_list[0]]
    
    route_timedata = travel_times_dict[index]
    
    time_mat = create_time_matrix(stops_list, route_timedata)
    pred_route = solve_TSP(stops_list, 1000*time_mat)
    
    pred_routes.append(pred_route)
    actual_routes.append(actual_route)
    stops_score.append(score(actual_route, pred_route, route_timedata))
    print("Solving", solve_counter, end='\r')
    solve_counter += 1

end = time.time()

In [41]:
base_df = test_df.assign(actual_routes=actual_routes, pred_routes=pred_routes,
                             stops_score=stops_score, time=end-start)[['station_code', 'route_score', 'actual_routes', 'pred_routes', 'stops_score', 'time']]
base_df

,station_code,route_score,actual_routes,pred_routes,stops_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,High,"[SV, LG, RD, AU, TU, BY, ZL, JH, IJ, ER, EV, V...","[SV, YV, VN, UV, UO, YI, FR, VI, XM, EG, PC, E...",0.063599,99.768958
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,High,"[RE, YJ, IT, QJ, PF, TL, EL, AQ, HW, VU, LH, I...","[RE, UT, IF, TX, KS, GS, OL, BL, QI, VQ, QO, A...",0.042564,99.768958
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,High,"[UX, BP, FM, WD, YK, HC, DH, DR, PG, MC, LI, U...","[UX, ER, FJ, PZ, WZ, GC, KY, UD, NC, WP, QS, D...",0.150547,99.768958
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,High,"[UZ, YM, VO, YR, DT, DQ, QT, BO, TM, IV, ZM, U...","[UZ, NT, UL, PV, AH, TG, HM, FO, ZI, XJ, IZ, T...",0.240476,99.768958
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,High,"[KV, HJ, NM, JV, BG, IF, XV, NU, CT, KW, UD, U...","[KV, HJ, NM, XV, IF, NU, CT, KW, AB, SR, TA, C...",0.041270,99.768958
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,High,"[DO, GC, GJ, GL, UQ, AE, RK, ML, ZB, LI, YR, T...","[DO, CV, BO, SY, VN, LC, HE, DL, EM, ZA, PQ, F...",0.086257,99.768958
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,High,"[BB, EO, OJ, JI, EX, ED, SZ, QG, SY, TP, TX, E...","[BB, VW, HB, IO, LG, MG, DE, MJ, ZT, AL, DK, Y...",0.112949,99.768958
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,High,"[CJ, TB, JV, GH, BB, KY, EF, BH, IB, ZN, AS, B...","[CJ, SV, AZ, RQ, NU, NZ, QD, KN, XV, YF, TC, A...",0.076588,99.768958
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,High,"[QP, BM, TY, OH, FZ, KR, OK, IL, XE, HW, OF, B...","[QP, IL, OK, KR, KH, FZ, VN, QH, JJ, VS, TH, O...",0.141102,99.768958
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,High,"[NZ, XU, KJ, XQ, ED, OI, XB, HG, FM, DK, FW, Z...","[NZ, WR, WC, QZ, RC, EQ, ZY, FA, JU, HK, NY, Z...",0.146493,99.768958


In [42]:
base_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\2702093360.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  base_df.mean()


stops_score     0.099599
time           99.768958
dtype: float64

### Stops routing; Distance-based zone sequence

In [59]:
dist_df

,station_code,sequences_of_zones,pred_sequences,AD,zone_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...","[0, F-4.3B, F-4.2B, F-4.1B, F-4.2C, F-4.3C, F-...",93.333333,0.041023,2.405951
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-...","[0, D-5.1B, D-5.1C, D-5.2C, D-5.3C, D-5.3D, D-...",72.727273,0.008896,2.405951
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...","[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-7.2A, A-...",78.260870,0.025279,2.405951
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2...","[0, B-12.1H, B-12.1G, B-12.3E, B-12.3G, B-12.2...",75.000000,0.055791,2.405951
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...","[0, D-1.2H, D-1.3J, D-1.2J, D-2.1J, D-1.1J, D-...",75.000000,0.019273,2.405951
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-...","[0, H-1.3B, G-1.3A, G-1.3B, G-1.1B, G-1.2B, G-...",76.923077,0.033160,2.405951
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1...","[0, C-12.2H, C-12.1H, C-12.3G, C-12.2G, C-12.1...",45.000000,0.090060,2.405951
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...","[0, J-20.1C, J-20.1B, J-20.3B, J-20.2B, J-20.2...",60.000000,0.112222,2.405951
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...","[0, A-7.1B, A-7.3C, A-7.3D, A-7.2D, A-7.2C, A-...",72.727273,0.066293,2.405951
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3...","[0, E-16.1A, G-16.1A, G-16.2A, G-16.3A, G-16.3...",87.500000,0.083720,2.405951


In [60]:
start = time.time()

stops_score = []
actual_routes = []
pred_routes = []
solve_counter = 0

for index, row in test_df.iterrows():
    stops_list = [*{k: v for k, v in sorted(row["actual"].items(), key=lambda item: item[1])}]
    actual_route = stops_list + [stops_list[0]]
    
    route_timedata = travel_times_dict[index]
    
    time_mat = create_time_matrix(stops_list, route_timedata)
    time_mat = time_mat/time_mat.sum(axis=1, keepdims=True) # normalize
    
#     pred_zone_seq = literal_eval(dist_df["pred_sequences"][index])
    pred_zone_seq = dist_df["pred_sequences"][index]
    
    # eliminate NoZones    
    stop_zone_dict = {stop: row["stops"][stop]['zone_id'] for stop in stops_list}
    for k, v in stop_zone_dict.items():
        if v is None:
            i = 1
            while stop_zone_dict[k] is None:
                shortest_dist = sorted(route_timedata[k].values())[i]
                for stop, dist_to_k in route_timedata[k].items():
                    if dist_to_k == shortest_dist:
                        stop_zone_dict[k] = row["stops"][stop]['zone_id']
                i += 1
    
    # zone penalty matrix
    zone_penalty_mat = np.zeros((len(stops_list), len(stops_list)))
    for i, source in enumerate(stops_list):
        for j, dest in enumerate(stops_list):
            zone_i = stop_zone_dict[source]
            zone_j = stop_zone_dict[dest]
            
            zone_diff = pred_zone_seq.index(zone_j)-pred_zone_seq.index(zone_i) # dest-source
            if zone_diff == 0:
                zone_penalty_mat[i][j] = 0 # same zone
            elif zone_diff == 1:
                zone_penalty_mat[i][j] = 1 # next zone
            else:
                zone_penalty_mat[i][j] = 2
                
    time_with_penalty_mat = np.zeros((len(stops_list), len(stops_list)))
    for i, source in enumerate(stops_list):
        for j, dest in enumerate(stops_list):
            time_with_penalty_mat[i][j] = time_mat[i][j] + zone_penalty_mat[i][j]
            
    pred_route = solve_TSP(stops_list, 10000*time_with_penalty_mat)
#     pred_route = solve_TSP_LS(stops_list, 1000*time_with_penalty_mat)
    
    pred_routes.append(pred_route)
    actual_routes.append(actual_route)
    stops_score.append(score(actual_route, pred_route, route_timedata))
    print("Solving", solve_counter, end='\r')
    solve_counter += 1
    
end = time.time()

In [61]:
stops_dist_df = test_df.assign(actual_routes=actual_routes, pred_routes=pred_routes,
                             stops_score=stops_score, time_st2=end-start)[['station_code', 'route_score', 'actual_routes', 'pred_routes', 'stops_score', 'time_st2']]
stops_dist_df.head()

,station_code,route_score,actual_routes,pred_routes,stops_score,time_st2
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,High,"[SV, LG, RD, AU, TU, BY, ZL, JH, IJ, ER, EV, V...","[SV, YV, BT, AF, EW, BE, PC, EG, SN, MO, PB, V...",0.045691,73.308735
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,High,"[RE, YJ, IT, QJ, PF, TL, EL, AQ, HW, VU, LH, I...","[RE, UT, IF, TX, KS, BW, JC, YE, NK, LJ, OO, O...",0.021499,73.308735
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,High,"[UX, BP, FM, WD, YK, HC, DH, DR, PG, MC, LI, U...","[UX, UW, GR, HY, NQ, IB, RC, ON, SA, VJ, YJ, E...",0.054704,73.308735
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,High,"[UZ, YM, VO, YR, DT, DQ, QT, BO, TM, IV, ZM, U...","[UZ, YJ, OQ, CF, ZC, BY, SI, OY, LI, FT, XI, J...",0.280275,73.308735
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,High,"[KV, HJ, NM, JV, BG, IF, XV, NU, CT, KW, UD, U...","[KV, IF, NU, KW, CT, UD, WW, BP, VF, DT, KD, V...",0.041216,73.308735


In [62]:
stops_dist_df = pd.concat([stops_dist_df, dist_df["zone_score"]], axis=1)

In [63]:
stops_dist_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\1745663541.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stops_dist_df.mean()


stops_score     0.061036
time_st2       73.308735
zone_score      0.058118
dtype: float64

### Stops routing; Markov

In [64]:
markov_all_df

,station_code,sequences_of_zones,pred_sequences,AD,zone_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,"[0, F-5.3D, F-5.2D, F-5.1D, F-4.1D, F-4.2D, F-...","[0, F-4.2B, F-4.1D, F-4.2D, F-4.3D, F-4.3C, F-...",33.333333,0.064731,2.550212
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,"[0, D-5.1E, D-5.1D, D-5.2D, D-5.3D, D-5.3C, D-...","[0, D-5.3B, D-5.2B, D-5.1B, D-5.1C, D-5.2C, D-...",100.000000,0.000000,2.550212
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,"[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...","[0, A-8.1C, A-8.1B, A-8.2B, A-8.3B, A-8.3A, A-...",0.000000,0.000000,2.550212
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,"[0, B-13.1J, B-13.2J, B-13.3J, B-13.3H, B-13.2...","[0, B-12.1H, B-12.1G, B-12.2G, B-12.3G, B-12.3...",100.000000,0.017316,2.550212
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,"[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...","[0, D-2.3H, D-2.3J, D-2.2J, D-2.1J, D-1.1J, D-...",0.000000,0.000000,2.550212
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,"[0, H-1.3B, H-1.3A, H-1.2A, H-1.1A, G-1.1A, G-...","[0, H-1.1A, H-1.2A, H-1.3A, H-1.3B, G-1.2A, G-...",53.846154,0.014304,2.550212
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,"[0, C-12.3G, C-11.3E, C-11.3G, C-11.2G, C-11.1...","[0, C-11.3E, C-11.3G, C-11.2G, C-11.1G, C-11.1...",45.000000,0.037230,2.550212
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,"[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...","[0, J-19.1D, J-19.2D, J-19.3D, J-19.3C, J-19.2...",0.000000,0.000000,2.550212
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,"[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...","[0, A-8.2B, A-8.3B, A-8.3A, A-8.2A, A-8.1A, A-...",0.000000,0.000000,2.550212
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,"[0, G-16.2B, G-16.3A, E-16.1B, E-16.2B, G-16.3...","[0, E-16.1B, E-16.2B, E-16.3B, E-16.3A, E-16.2...",62.500000,0.095985,2.550212


In [65]:
start = time.time()

stops_score = []
actual_routes = []
pred_routes = []
solve_counter = 0

for index, row in test_df.iterrows():
    stops_list = [*{k: v for k, v in sorted(row["actual"].items(), key=lambda item: item[1])}]
    actual_route = stops_list + [stops_list[0]]
    
    route_timedata = travel_times_dict[index]
    
    time_mat = create_time_matrix(stops_list, route_timedata)
    time_mat = time_mat/time_mat.sum(axis=1, keepdims=True) # normalize
   
#     pred_zone_seq = literal_eval(markov_all_df["pred_sequences"][index])
    pred_zone_seq = markov_all_df["pred_sequences"][index]
    
    # eliminate NoZones    
    stop_zone_dict = {stop: row["stops"][stop]['zone_id'] for stop in stops_list}
    for k, v in stop_zone_dict.items():
        if v is None:
            i = 1
            while stop_zone_dict[k] is None:
                shortest_dist = sorted(route_timedata[k].values())[i]
                for stop, dist_to_k in route_timedata[k].items():
                    if dist_to_k == shortest_dist:
                        stop_zone_dict[k] = row["stops"][stop]['zone_id']
                i += 1
    
    # zone penalty matrix
    zone_penalty_mat = np.zeros((len(stops_list), len(stops_list)))
    for i, source in enumerate(stops_list):
        for j, dest in enumerate(stops_list):
            zone_i = stop_zone_dict[source]
            zone_j = stop_zone_dict[dest]
            
            zone_diff = pred_zone_seq.index(zone_j)-pred_zone_seq.index(zone_i) # dest-source
            if zone_diff == 0:
                zone_penalty_mat[i][j] = 0 # same zone
            elif zone_diff == 1:
                zone_penalty_mat[i][j] = 1 # next zone
            else:
                zone_penalty_mat[i][j] = 2
                
    time_with_penalty_mat = np.zeros((len(stops_list), len(stops_list)))
    for i, source in enumerate(stops_list):
        for j, dest in enumerate(stops_list):
            time_with_penalty_mat[i][j] = time_mat[i][j] + zone_penalty_mat[i][j]
            
    pred_route = solve_TSP(stops_list, 10000*time_with_penalty_mat)
#     pred_route = solve_TSP_LS(stops_list, 1000*time_with_penalty_mat)
    
    pred_routes.append(pred_route)
    actual_routes.append(actual_route)
    stops_score.append(score(actual_route, pred_route, route_timedata))
    print("Solving", solve_counter, end='\r')
    solve_counter += 1
    
end = time.time()

In [66]:
stops_markov_all_df = test_df.assign(actual_routes=actual_routes, pred_routes=pred_routes,
                             stops_score=stops_score, time=end-start)[['station_code', 'route_score', 'actual_routes', 'pred_routes', 'stops_score', 'time']]
stops_markov_all_df.head()

,station_code,route_score,actual_routes,pred_routes,stops_score,time
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,High,"[SV, LG, RD, AU, TU, BY, ZL, JH, IJ, ER, EV, V...","[SV, YV, BT, AF, EW, BE, PC, EG, SN, MO, PB, V...",0.061980,73.424205
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,High,"[RE, YJ, IT, QJ, PF, TL, EL, AQ, HW, VU, LH, I...","[RE, UT, IF, TX, KS, BW, JC, YE, NK, LJ, OO, O...",0.021615,73.424205
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,High,"[UX, BP, FM, WD, YK, HC, DH, DR, PG, MC, LI, U...","[UX, UW, GR, HY, NQ, IB, ON, RC, SA, VJ, UJ, N...",0.025253,73.424205
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,High,"[UZ, YM, VO, YR, DT, DQ, QT, BO, TM, IV, ZM, U...","[UZ, NQ, RY, AC, TZ, PA, GK, VV, YA, DH, TK, B...",0.179126,73.424205
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,High,"[KV, HJ, NM, JV, BG, IF, XV, NU, CT, KW, UD, U...","[KV, XV, HJ, NM, JV, BG, IF, NU, CT, KW, UD, W...",0.005411,73.424205


In [68]:
stops_markov_all_df = pd.concat([stops_markov_all_df, markov_all_df["zone_score"]], axis=1)
stops_markov_all_df

,station_code,route_score,actual_routes,pred_routes,stops_score,time,zone_score,zone_score
RouteID_e164e0c7-979e-4645-83df-ea3f213578de,DLA3,High,"[SV, LG, RD, AU, TU, BY, ZL, JH, IJ, ER, EV, V...","[SV, YV, BT, AF, EW, BE, PC, EG, SN, MO, PB, V...",0.061980,73.424205,0.064731,0.064731
RouteID_e17b363d-c2d7-45fd-9829-2c631a335174,DSE4,High,"[RE, YJ, IT, QJ, PF, TL, EL, AQ, HW, VU, LH, I...","[RE, UT, IF, TX, KS, BW, JC, YE, NK, LJ, OO, O...",0.021615,73.424205,0.000000,0.000000
RouteID_e18f4d42-a31f-4b0f-b056-ecb8dae9b311,DLA3,High,"[UX, BP, FM, WD, YK, HC, DH, DR, PG, MC, LI, U...","[UX, UW, GR, HY, NQ, IB, ON, RC, SA, VJ, UJ, N...",0.025253,73.424205,0.000000,0.000000
RouteID_e1a4bf8a-a54b-41b4-bf38-266cc87a9990,DAU1,High,"[UZ, YM, VO, YR, DT, DQ, QT, BO, TM, IV, ZM, U...","[UZ, NQ, RY, AC, TZ, PA, GK, VV, YA, DH, TK, B...",0.179126,73.424205,0.017316,0.017316
RouteID_e1b827ff-76b1-4788-9f41-527e481461de,DSE4,High,"[KV, HJ, NM, JV, BG, IF, XV, NU, CT, KW, UD, U...","[KV, XV, HJ, NM, JV, BG, IF, NU, CT, KW, UD, W...",0.005411,73.424205,0.000000,0.000000
RouteID_e1c24ffd-ca8e-4666-a8ae-25e9893ecba0,DLA4,High,"[DO, GC, GJ, GL, UQ, AE, RK, ML, ZB, LI, YR, T...","[DO, CV, VN, LC, BO, SY, HB, BD, FD, PQ, NO, O...",0.035156,73.424205,0.014304,0.014304
RouteID_e1fcf551-00f4-4401-b4cb-5e264cea6561,DBO3,High,"[BB, EO, OJ, JI, EX, ED, SZ, QG, SY, TP, TX, E...","[BB, VW, HB, IO, LG, HG, MU, LW, IM, ST, DK, A...",0.038229,73.424205,0.037230,0.037230
RouteID_e20571c7-de1a-47f1-98e7-fded3e542bf7,DLA9,High,"[CJ, TB, JV, GH, BB, KY, EF, BH, IB, ZN, AS, B...","[CJ, TB, JV, GH, KY, IB, BH, EF, BB, ZN, AS, B...",0.008061,73.424205,0.000000,0.000000
RouteID_e245c849-5520-4b34-a4fe-55ef8237cc59,DLA3,High,"[QP, BM, TY, OH, FZ, KR, OK, IL, XE, HW, OF, B...","[QP, IL, OK, KR, FZ, HW, OF, LK, BK, UP, AT, R...",0.019797,73.424205,0.000000,0.000000
RouteID_e246533a-8167-4c0d-8c80-783847125b1b,DLA4,High,"[NZ, XU, KJ, XQ, ED, OI, XB, HG, FM, DK, FW, Z...","[NZ, WR, WC, AQ, SO, ZY, EQ, TV, DJ, VK, QQ, S...",0.086983,73.424205,0.095985,0.095985


In [69]:
stops_markov_all_df.mean()

C:\Users\gebruiker\AppData\Local\Temp\ipykernel_10596\4201308935.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stops_markov_all_df.mean()


stops_score     0.048780
time           73.424205
zone_score      0.025556
zone_score      0.025556
dtype: float64